# SparkSession 생성

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# fakefriends.csv로딩

In [3]:
# RDD 에서 로딩하는 방법으로 불러오기
directory="C:\\Users\\daesi\\Downloads\\빅데이터 소스코드\\소스코드\\study_spark\\data"
filename  = "fakefriends.csv"

lines = spark.sparkContext.textFile(f"file:///{directory}\\{filename}")
lines.collect()[:3]


['0,Will,33,385', '1,Jean-Luc,26,2', '2,Hugh,55,221']

In [4]:
def mapper(line):
    fields = line.split(",")
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), age=int(fields[2]), numFriends=int(fields[3]))

# rdd의 map을 이용해서 Row 형식으로 모두 바꿔준다.
people = lines.map(mapper)
people.collect()[:3]

[Row(ID=0, name="b'Will'", age=33, numFriends=385),
 Row(ID=1, name="b'Jean-Luc'", age=26, numFriends=2),
 Row(ID=2, name="b'Hugh'", age=55, numFriends=221)]

매핑된 people RDD를 활용해 데이터 프레임 생성

In [5]:
schemaPeople = spark.createDataFrame(people).cache()

schemaPeople.createOrReplaceTempView("people")
schemaPeople.show()

+---+-----------+---+----------+
| ID|       name|age|numFriends|
+---+-----------+---+----------+
|  0|    b'Will'| 33|       385|
|  1|b'Jean-Luc'| 26|         2|
|  2|    b'Hugh'| 55|       221|
|  3|  b'Deanna'| 40|       465|
|  4|   b'Quark'| 68|        21|
|  5|  b'Weyoun'| 59|       318|
|  6|  b'Gowron'| 37|       220|
|  7|    b'Will'| 54|       307|
|  8|  b'Jadzia'| 38|       380|
|  9|    b'Hugh'| 27|       181|
| 10|     b'Odo'| 53|       191|
| 11|     b'Ben'| 57|       372|
| 12|   b'Keiko'| 54|       253|
| 13|b'Jean-Luc'| 56|       444|
| 14|    b'Hugh'| 43|        49|
| 15|     b'Rom'| 36|        49|
| 16|  b'Weyoun'| 22|       323|
| 17|     b'Odo'| 35|        13|
| 18|b'Jean-Luc'| 45|       455|
| 19|  b'Geordi'| 60|       246|
+---+-----------+---+----------+
only showing top 20 rows



# sql을 이용해 데이터 조회
`createOrReplaceTempView`를 이용해 등록된 테이블에 쿼리를 사용할 수 있습니다.

In [6]:
# 쿼리를 사용해서 13 ~ 19세의 모든 정보를 조회
query = """
SELECT *
  FROM people
 WHERE age >= 13 AND age <= 19
"""
spark.sql(query).show()

+---+----------+---+----------+
| ID|      name|age|numFriends|
+---+----------+---+----------+
| 21|  b'Miles'| 19|       268|
| 52|b'Beverly'| 19|       269|
| 54|  b'Brunt'| 19|         5|
|106|b'Beverly'| 18|       499|
|115|  b'Dukat'| 18|       397|
|133|  b'Quark'| 19|       265|
|136|   b'Will'| 19|       335|
|225|   b'Elim'| 19|       106|
|304|   b'Will'| 19|       404|
|341|   b'Data'| 18|       326|
|366|  b'Keiko'| 19|       119|
|373|  b'Quark'| 19|       272|
|377|b'Beverly'| 18|       418|
|404| b'Kasidy'| 18|        24|
|409|    b'Nog'| 19|       267|
|439|   b'Data'| 18|       417|
|444|  b'Keiko'| 18|       472|
|492|  b'Dukat'| 19|        36|
|494| b'Kasidy'| 18|       194|
+---+----------+---+----------+



In [7]:
schemaPeople.groupBy("age").count().orderBy("age").show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [8]:
spark.stop()